In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import math
import sqlite3
import pandas as pd
import numpy as np
import urllib
from lxml import etree
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sqlite3
import pickle

In [2]:
#initial request
url = 'https://www.basketball-reference.com/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
#find links for individual page
urls = []
for i in soup.find_all('tr', class_="full_table"):
    urls.append(url + i.find('a', href=True)['href'])
urls

['https://www.basketball-reference.com//teams/MIL/2020.html',
 'https://www.basketball-reference.com//teams/TOR/2020.html',
 'https://www.basketball-reference.com//teams/BOS/2020.html',
 'https://www.basketball-reference.com//teams/MIA/2020.html',
 'https://www.basketball-reference.com//teams/IND/2020.html',
 'https://www.basketball-reference.com//teams/PHI/2020.html',
 'https://www.basketball-reference.com//teams/BRK/2020.html',
 'https://www.basketball-reference.com//teams/ORL/2020.html',
 'https://www.basketball-reference.com//teams/WAS/2020.html',
 'https://www.basketball-reference.com//teams/CHO/2020.html',
 'https://www.basketball-reference.com//teams/CHI/2020.html',
 'https://www.basketball-reference.com//teams/NYK/2020.html',
 'https://www.basketball-reference.com//teams/DET/2020.html',
 'https://www.basketball-reference.com//teams/ATL/2020.html',
 'https://www.basketball-reference.com//teams/CLE/2020.html',
 'https://www.basketball-reference.com//teams/LAL/2020.html',
 'https:

In [4]:
with open('listofurls.pkl', 'wb') as f:
    pickle.dump(urls, f)

In [5]:
with open('listofurls.pkl', 'rb') as f:
    mynewlist = pickle.load(f)

In [6]:
option = webdriver.ChromeOptions()
option.add_argument(" — incognito")
driver = webdriver.Chrome(executable_path='/Users/Anubhav/Desktop/Flatiron School/QCstats/chromedriver', options=option)

In [127]:
all_individuals = pd.DataFrame()
for url in range(len(urls)):
    #go to team site
    driver.get(urls[url])
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "all_roster")))

    #create dataframes for each table
    df0 = pd.read_html(driver.find_element_by_css_selector('table[id="per_game"]').get_attribute('outerHTML'))[0]
    df1 = pd.read_html(driver.find_element_by_css_selector('table[id="totals"]').get_attribute('outerHTML'))[0]
    df2 = pd.read_html(driver.find_element_by_css_selector('table[id="per_minute"]').get_attribute('outerHTML'))[0]
    df3 = pd.read_html(driver.find_element_by_css_selector('table[id="per_poss"]').get_attribute('outerHTML'))[0]
    df4 = pd.read_html(driver.find_element_by_css_selector('table[id="advanced"]').get_attribute('outerHTML'))[0]
    df5 = pd.read_html(driver.find_element_by_css_selector('table[id="shooting"]').get_attribute('outerHTML'))[0]
    df6 = pd.read_html(driver.find_element_by_css_selector('table[id="pbp"]').get_attribute('outerHTML'))[0]

    #change column names of all player tables
    df0.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MP/G', 'FGMade/G', 'FGA/G', 'FGPerc',
                   'ThreeMade/G', 'ThreeA/G', 'ThreePerc','TwoMade/G', 'TwoA/G',
                   'TwoPerc', 'eFGPerc', 'FTMade/G', 'FTA/G', 'FTPerc', 'ORB/G',
                   'DRB/G', 'TRB/G','AST/G', 'STL/G', 'BLK/G', 'TOV/G', 'PF/G', 'PTS/G']
    df1.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MPTotal', 'FGMadeTotal',
                   'FGATotal', 'FGPerc', 'ThreeMadeTotal', 'ThreeATotal', 'ThreePerc',
                   'TwoMadeTotal', 'TwoATotal', 'TwoPerc', 'eFGPerc', 'FTMadeTotal',
                   'FTATotal', 'FTPerc', 'ORBTotal', 'DRBTotal', 'TRBTotal',
                   'ASTTotal', 'STLTotal', 'BLKTotal', 'TOVTotal', 'PFTotal', 'PTSTotal']
    df1.drop(df1.tail(1).index, inplace=True)
    df2.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MP/THREESIX', 'FGMade/THREESIX',
                   'FGA/THREESIX', 'FGPerc', 'ThreeMade/THREESIX', 'ThreeA/THREESIX',
                   'ThreePerc','TwoMade/THREESIX', 'TwoA/THREESIX', 'TwoPerc',
                   'FTMade/THREESIX', 'FTA/THREESIX', 'FTPerc', 'ORB/THREESIX', 'DRB/THREESIX',
                   'TRB/THREESIX', 'AST/THREESIX', 'STL/THREESIX', 'BLK/THREESIX',
                   'TOV/THREESIX', 'PF/THREESIX', 'PTS/THREESIX']
    df3.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MP/Hund', 'FGMade/Hund', 'FGA/Hund',
                   'FGPerc', 'ThreeMade/Hund', 'ThreeA/Hund', 'ThreePerc',
                   'TwoMade/Hund', 'TwoA/Hund', 'TwoPerc', 'FTMade/Hund', 'FTA/Hund',
                   'FTPerc', 'ORB/Hund', 'DRB/Hund', 'TRB/Hund', 'AST/Hund', 'STL/Hund',
                   'BLK/Hund', 'TOV/Hund', 'PF/Hund', 'PTS/Hund', 'Blank', 'ORTG', 'DRTG']
    df4.columns = ['Rk', 'Name', 'Age', 'G', 'MPTotal', 'PER', 'TSPerc', 'PercThreeA',
                   'PercFTA/FG', 'PercORB', 'PercDRB', 'PercTRB', 'PercAST', 'PercSTL',
                   'PercBLK', 'PercTOV', 'USG', 'Blank1', 'OWS', 'DWS', 'WS', 'WS/48',
                   'Blank2', 'OBPM', 'DBPM', 'BPM', 'VORP']
    df5.columns = ['Rk', 'Name', 'Age', 'G', 'MPTotal', 'FGPerc', 'AvgDis',
                   'PercTwoA', 'PercZeroThreefeetA','PercThreefeetTenA', 'PercTenSixTA',
                   'PercSixTThreeA', 'PercThreeA', 'TwoPerc', 'ZeroThreePerc',
                   'ThreefeetTenPerc', 'TenSixTPerc', 'SixTThreePerc', 'ThreePerc',
                   'PercTwoAstd', 'PercFGADunks', 'MadeDunks', 'PercThreeAstd',
                   'Perc3PACorner', 'ThreePercCorner', 'HeavesA', 'MadeHeaves']
    df6.columns = ['Rk', 'Name', 'Age', 'G', 'MP', 'PercasPG', 'PercasSG',
                   'PercasSF', 'PercasPF', 'PercasC', 'PMP100OnCourt', 'NPMP100',
                   'TOVBadPass', 'TOVLostBall', 'ShootFoulsC', 'OFoulsC',
                   'ShootFoulsD', 'OFoulsD', 'PGA', 'FAAnd1', 'FABlkd']

    #drop uncessary columns from each dataframe
    for table in range(1,7):
        checklist = []
        for x in range(0,table):
            exec(f"checklist.extend(list(df{x}.columns))")
            checklist.remove('Name')
        exec(f"df{table}.drop([col for col in df{table}.columns if col in checklist], axis = 1, inplace = True)")
        exec(f"blank = [col for col in df{table} if col.startswith('Blank')]")
        if not blank:
            pass
        else:
            exec(f"df{table}.drop(blank, axis = 1, inplace = True)")

    #full outer join all dataframes
    for x in range(0,7):
        exec(f"df{x}.set_index('Name', inplace = True)")
    result = pd.concat([df0, df1, df2, df3, df4, df5, df6], axis=1, sort = True)
    result.drop('Rk', axis = 1, inplace = True)
    result.reset_index(inplace = True)
    result.rename(columns = {'index': 'Name'}, inplace = True)

    #add TeamName column
    for idx, x in enumerate(result.Name):
        result.loc[idx, 'Team'] = urls[url][44:47]
    
    #add rows to pre-existing dataframe
    all_individuals = pd.concat([all_individuals, result], ignore_index=True)

In [129]:
all_individuals.to_pickle("individualstats.pkl")

In [130]:
all_individuals = pd.read_pickle("individualstats.pkl")

In [131]:
all_individuals

,Name,Age,G,GS,MP/G,FGMade/G,FGA/G,FGPerc,ThreeMade/G,ThreeA/G,...,TOVBadPass,TOVLostBall,ShootFoulsC,OFoulsC,ShootFoulsD,OFoulsD,PGA,FAAnd1,FABlkd,Team
0,Brook Lopez,31,61,60,26.6,4.0,9.4,0.427,1.4,4.7,...,22,7,96,18,61,3,240,13,20,MIL
1,D.J. Wilson,23,31,0,9.0,1.3,3.3,0.396,0.5,1.8,...,6,1,17,1,5,3,54,1,3,MIL
2,Donte DiVincenzo,23,59,22,23.1,3.5,7.7,0.462,1.3,3.7,...,40,20,40,8,30,17,317,2,23,MIL
3,Dragan Bender,22,7,0,13.0,1.4,3.0,0.476,0.6,1.3,...,4,1,13,0,2,0,21,1,4,MIL
4,Eric Bledsoe,30,56,56,27.2,5.7,11.8,0.482,1.3,3.6,...,71,36,37,23,98,15,733,31,37,MIL
5,Ersan İlyasova,32,56,8,16.0,2.4,5.2,0.472,0.8,2.2,...,19,4,46,5,31,17,117,7,13,MIL
6,Frank Mason,25,6,0,8.8,0.8,3.2,0.263,0.3,1.7,...,2,4,3,0,2,0,29,0,3,MIL
7,George Hill,33,52,0,21.2,3.4,6.5,0.530,1.4,2.9,...,26,11,34,4,24,16,361,4,20,MIL
8,Giannis Antetokounmpo,25,57,57,30.9,10.9,20.0,0.547,1.5,4.8,...,60,54,44,55,276,10,836,86,64,MIL
9,Khris Middleton,28,55,52,30.1,7.7,15.5,0.499,2.4,5.8,...,54,32,43,19,73,10,519,15,33,MIL
